$$\text{Deterministic and Hybrid Solvers}$$

Requirements: Imports below, and the classes/functions from the neural network .ipynb file. I will try to put these into a .py file for easier imports, else can copy/paste in the NN file contents.

Single-point solver: the algorithm checks local mine counts to determine if the number of adjacent squares = number of remaining mines, or if number of mines = number of known mines in adjacent squares. If the former, the algorithm identifies a mine, and if the latter, the algorithm digs. In cases with no clear mine or safe square, it guesses.

In [ ]:
# Imports: NumPy, Matplotlib, collections

import numpy as np
import matplotlib.pyplot as plt
from collections import deque

# Picks first move (which is safe); this function is reused for the NN logic and the next (CSP) solver for consistency when comparing. 
def first_click_index(env):
    i, j = env.r // 2, env.c // 2  
    return env._flat(i, j)

# Returns all neighbor coordinates using "yield"
def neighbors(env, i, j):
    for x, y in env._neighbors(i, j):
        yield x, y

# Returns all neighbors that are still covered
def covered_neighbors(env, i, j):
    return [(x, y) for x, y in neighbors(env, i, j) if env.mask[x, y] == 1 and env.board[x, y] != -1]

# Gives number of nonmine revealed cell
def revealed_number(env, i, j):
    return int(env._counts[i, j])

# As in the GNN, this function pulls all of the cells that would be visible to the player, i.e. places you can use logic to find mines.
def frontier_cells(env):
    cov = (env.mask == 1)
    rev = ~cov
    F = set()
    for i in range(env.r):
        for j in range(env.c):
            if rev[i, j] and env._counts[i, j] > 0:
                for x, y in neighbors(env, i, j):
                    if env.mask[x, y] == 1:
                        F.add((x, y))
    return F

# Body of the single-point solver
def solve_single_point(env, rng):
    # Reset environment to replay and compile winrates
    env.reset()
    obs, r, done, info = env.step(first_click_index(env))
    flags = set()
    while not done:
        progress = False
    # Finds mines and safe cells, whereas the NN algo only digs
        to_flag = set()
        to_click = set()

    # Loop over all cells
        for i in range(env.r):
            for j in range(env.c):
    # Check if revealed to user and in frontier
                if env.mask[i, j] == 0 and env._counts[i, j] > 0:
                    cov = [(x, y) for x, y in neighbors(env, i, j) if env.mask[x, y] == 1 and (x, y) not in flags]
                    f_cnt = sum((x, y) in flags for x, y in neighbors(env, i, j))
                    need = env._counts[i, j] - f_cnt
                    if need < 0:  
                        continue
                    if need == 0 and cov:
    # Checks if all covered neighbors are safe
                        to_click.update(cov)
                    elif need == len(cov) and len(cov) > 0:
    # Checks if all covered neighbors include mines
                        to_flag.update(cov)
    # Actions: flag mines and click nonmines
        if to_flag:
            flags.update(to_flag)
            progress = True
        if to_click:
            for (x, y) in list(to_click):
                if env.mask[x, y] == 1:
                    obs, r, done, info = env.step(env._flat(x, y))
                    if done:
                        break
            progress = True
            if done:
                break
    # Random guess if no deterministic moves, or break the loop if there is an error and no moves are available.
        if not progress:
            F = [xy for xy in frontier_cells(env) if xy not in flags]
            if not F:
                F = [(i, j) for i in range(env.r) for j in range(env.c) if env.mask[i, j] == 1 and (i, j) not in flags]
            if not F:
                break
            x, y = F[rng.integers(len(F))]
            obs, r, done, info = env.step(env._flat(x, y))

    return info.get("result") == "win"

CSP Solver: Based on the algorithm described here: https://www.cs.toronto.edu/~cvs/minesweeper/minesweeper.pdf. I tried to implement this to the best of my ability, but there are likely some issues or potential improvements to get a better CSP solver performance. 
\
The CSP solver reads the board as a matrix of booleans (True/1 if mine, else False/0). Then, it constructs a bunch of linear constraints of the form: $x_1 + x_2 + ... = \text{need}$ where $x_i$ is the probability cell $i$ on a flattened index is a mine, and "need" is the number of mines in adjacent squares around a revealed cell.  
\
Then, the algo finds these constraints for each frontier cell, and creates an adjacency graph between elements that appear in a common constraint equation. 
\
The functions 'feasible()' and 'backtrack()' then go through each square from most to least constrained (appears in the fewest constraints), and assign 0/1 values to each cell. Solutions that are infeasible because some constraints become impossible are pruned out, while solutions where there is at least one viable board satisfying all outstanding constraints are feasible. 
\
For feasible constraints, cells are ranked by the quotient: [# adjacent mines/# solutions], and the solver digs the cell with the lowest probability of a mine based on that quotient. Then it recomputes for the updated frontier, and repeats.

In [ ]:
# Goes through the board and adds constraints via tuples of ([constraint vars], need)

def build_constraints(env, vars_set):
    """Return constraints over vars (indexed 0..V-1) for all numbered border cells."""
    idx = {xy: k for k, xy in enumerate(vars_set)}
    constraints = []
    for i in range(env.r):
        for j in range(env.c):
            if env.mask[i, j] == 0 and env._counts[i, j] > 0:
                neigh_cov = [(x, y) for x, y in neighbors(env, i, j) if env.mask[x, y] == 1]
                var_idx = [idx[xy] for xy in neigh_cov if xy in idx]
                if not var_idx:
                    continue
                need = env._counts[i, j]
                # Subtract already-flagged if you maintain flags; here we do not place flags persistently
                constraints.append((var_idx, need))
    return constraints

# Creates connected components of frontier cells via shared constraints

def componentize(env, F):
    """Split frontier into connected components via shared numbered neighbors."""
    F_list = list(F)
    idx = {xy: k for k, xy in enumerate(F_list)}
    adj = [set() for _ in F_list]
    for i in range(env.r):
        for j in range(env.c):
            if env.mask[i, j] == 0 and env._counts[i, j] > 0:
                comp = [(x, y) for x, y in neighbors(env, i, j) if env.mask[x, y] == 1 and (x, y) in idx]
                for a in comp:
                    for b in comp:
                        if a != b:
                            adj[idx[a]].add(idx[b])
    comps = []
    seen = set()
    for k in range(len(F_list)):
        if k in seen:
            continue
    # Use a double queue to backtrack through variables
        q = deque([k])
        seen.add(k)
        cur = [k]
        while q:
            u = q.popleft()
            for v in adj[u]:
                if v not in seen:
                    seen.add(v)
                    q.append(v)
                    cur.append(v)
        comps.append([F_list[t] for t in cur])
    return comps

# Main part: checks feasibility, prunes, and backtracks to count solutions
def enumerate_solutions(variables, constraints, limit=100000):
    """
    variables: list of variable ids [0,...V-1]
    constraints: list of (vars_idx_list, sum_needed)
    Return: count_solutions, mine_counts_per_var (length V)
    """
    V = len(variables)
    cons_vars = [list(vs) for (vs, s) in constraints]
    cons_need = [int(s) for (vs, s) in constraints]

    order = sorted(range(V), key=lambda v: sum(v in vs for vs in cons_vars), reverse=True)  # heuristic
    assign = [None] * V
    sol_count = 0
    mine_count = np.zeros(V, dtype=np.int64)
    var_to_cons = [[] for _ in range(V)]
    for ci, vs in enumerate(cons_vars):
        for v in vs:
            var_to_cons[v].append(ci)

    def feasible(ci):
        """Check quick bounds for constraint ci."""
        vs = cons_vars[ci]
        need = cons_need[ci]
        min_left = 0
        max_left = 0
        for v in vs:
            if assign[v] is None:
                max_left += 1
            elif assign[v] == 1:
                need -= 1
        return 0 <= need <= max_left

    def backtrack(t):
        nonlocal sol_count
        if sol_count >= limit:
            return
        if t == V:
            for ci, vs in enumerate(cons_vars):
                need = cons_need[ci]
                s = sum(assign[v] == 1 for v in vs)
                if s != need:
                    return
            sol_count += 1
            for v in range(V):
                if assign[v] == 1:
                    mine_count[v] += 1
            return
        v = order[t]
        for val in (0, 1):
            assign[v] = val
            ok = True
            for ci in var_to_cons[v]:
                if not feasible(ci):
                    ok = False
                    break
            if ok:
                backtrack(t + 1)
            assign[v] = None

    for ci in range(len(cons_vars)):
        if not feasible(ci):
            return 0, mine_count
    backtrack(0)
    return sol_count, mine_count

def csp_step(env, rng):
    """One CSP-driven step. Returns (moved, done)."""
    F = frontier_cells(env)
    if not F:
    # Clicks random if no frontier. Else, clicks all minimum mine probability cell.
        cov = [(i, j) for i in range(env.r) for j in range(env.c) if env.mask[i, j] == 1]
        if not cov:
            return False, True
        x, y = cov[rng.integers(len(cov))]
        _, _, done, _ = env.step(env._flat(x, y))
        return True, done
    
    # In some cases there are multiple guaranteed nonmines -- clicks those at once
    best_xy = None
    best_p = 1.1
    for comp in componentize(env, F):
        vars_list = list(comp)
        constraints = []
        # Build constraints referencing only comp vars
        idx = {xy: k for k, xy in enumerate(vars_list)}
        for i in range(env.r):
            for j in range(env.c):
                if env.mask[i, j] == 0 and env._counts[i, j] > 0:
                    neigh = [(x, y) for x, y in neighbors(env, i, j) if env.mask[x, y] == 1 and (x, y) in idx]
                    if not neigh:
                        continue
                    constraints.append(([idx[xy] for xy in neigh], env._counts[i, j]))
        if not constraints:
            x, y = comp[rng.integers(len(comp))]
            _, _, done, _ = env.step(env._flat(x, y))
            return True, done

        sol_count, mine_count = enumerate_solutions(vars_list, constraints, limit=200000)
        if sol_count > 0:
            probs = mine_count / sol_count
            # Safe cell check
            safe_idxs = np.where(probs == 0)[0]
            if safe_idxs.size > 0:
                for si in safe_idxs:
                    x, y = vars_list[int(si)]
                    if env.mask[x, y] == 1:
                        _, _, done, _ = env.step(env._flat(x, y))
                        if done:
                            return True, True
                return True, False
            k = int(np.argmin(probs))
            if probs[k] < best_p:
                best_p = float(probs[k])
                best_xy = vars_list[k]
        else:
            x, y = comp[rng.integers(len(comp))]
            _, _, done, _ = env.step(env._flat(x, y))
            return True, done

    if best_xy is not None:
        x, y = best_xy
        _, _, done, _ = env.step(env._flat(x, y))
        return True, done

    # Fallback: random click
    cov = [(i, j) for i in range(env.r) for j in range(env.c) if env.mask[i, j] == 1]
    if cov:
        x, y = cov[rng.integers(len(cov))]
        _, _, done, _ = env.step(env._flat(x, y))
        return True, done
    return False, True

def solve_csp(env, rng):
    env.reset()
    _, _, done, info = env.step(first_click_index(env))
    while not done:
        moved, done = csp_step(env, rng)
        if not moved:
            break
    return info.get("result") == "win" or (done and getattr(env, "explosion", False) is False and np.all(env.mask[env.board != -1] == 0))

$$\text{Evaluation}$$

In [ ]:
# Checks solver winrates, works for CSP + SP + NN.

def eval_solver_over_mines(solver_fn, mines_list, rows=10, cols=10, trials=200, base_seed=0):
    # Uniform rng/seeds; good to verify not using the seeds the NN trained on here.
    rng_master = np.random.default_rng(base_seed)
    seeds = rng_master.integers(0, 2**31-1, size=trials, dtype=np.int64)
    wr = []
    for m in mines_list:
        wins = 0
    # Get env and apply the solver function for the agent; must output True.
        for sd in seeds:
            env = MinesweeperEnv(rows=rows, cols=cols, mines=int(m), seed=int(sd))
            rng = np.random.default_rng(sd)
            wins += 1 if solver_fn(env, rng) else 0
        wr.append(wins / trials)
    return np.array(wr, dtype=float)
